In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))
from policy import ConvNet, ResNet
from concepts import DynamicConcepts, linear_regression
import absl.logging
import numpy as np
from env import gogame
import tensorflow as tf
from utils import concept_folder_setup_and_score

absl.logging.set_verbosity(absl.logging.ERROR)

# Set memory growth
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

session_name = "falcon"
model_name = "net"
board_size = 7
board_name = f'{board_size}x{board_size}'
agents_to_sample = [0, 10, 20, 60, 100, 500]
resnet = True
random_subpar_rollout = False

model_type = "resnet" if resnet else "convnet"

# Mcts simulations
simulations = 2000

full_model_path = f"../models/saved_sessions/{model_type}/board_size_{board_size}/{session_name}/"

# Search for the model with the most seen games and use it for mcts simulations
folders = os.listdir(full_model_path)

# Sort the folders by the number in the name
sorted_folders = sorted(folders, key=lambda x: int(x.split('_')[-1].strip('.keras')))

# Get the last folder
mcts_model_path = full_model_path + sorted_folders[-1]

init_state, concept_type_single, concept_name = DynamicConcepts.ladder(board_size)

print("Initial state:\n", gogame.str(init_state))

dynamic_concept = DynamicConcepts(init_state, simulations, board_size, concept_type_single, mcts_model_path, random_subpar_rollout, resnet)

Initial state:
 	0 1 2 3 4 5 6 
0	╔═╤═╤═╤═╤═╤═╗
1	╟─┼─┼─┼─┼─┼─╢
2	╟─┼─┼─○─●─┼─╢
3	╟─┼─○─●─●─○─╢
4	╟─○─●─●─○─┼─╢
5	○─●─●─○─┼─┼─╢
6	╚═○═○═╧═╧═╧═╝
	Turn: BLACK, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 10, White Area: 7

INFO:tensorflow:Assets written to: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmpt12bom58/assets


INFO:tensorflow:Assets written to: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmpt12bom58/assets
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
positive_cases = []
negative_cases = []

pos, neg = dynamic_concept.generate_cases()

positive_cases.extend(pos)
negative_cases.extend(neg)

# Create numpy arrays
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

In [5]:
# Print shapes
print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (11, 5, 7, 7)
Negative cases:  (27, 5, 7, 7)


In [236]:
# If larger than 50, do not visualize the tree do to the size (crashes the kernel)
if simulations <= 100:
    graph = dynamic_concept.view_tree()

    graph.render('./visualization/images/tree', view=True)

In [36]:
# Test if there are any duplicates accross the two sets
for i in range(len(positive_cases)):
    for j in range(len(negative_cases)):
        if np.array_equal(positive_cases[i], negative_cases[j]):
            print("Duplicate found!")
            print(positive_cases[i])
            print(negative_cases[j])
            break

In [6]:
# Find the number of any duplicates within the positive cases
duplicate_count = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in positive cases: ", duplicate_count)

# Find the number of any duplicates within the negative cases
duplicate_count = 0
for i in range(len(negative_cases)):
    for j in range(i + 1, len(negative_cases)):
        if np.array_equal(negative_cases[i], negative_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in negative: ", duplicate_count)

Duplicate count in positive cases:  0
Duplicate count in negative:  0


In [ ]:
# Print all the positive cases
for i in range(len(positive_cases)):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print(gogame.str(positive_cases[i], nn_format=True))
    print()

In [ ]:
# Print all the negative cases
for i in range(len(negative_cases)):
    print(f"Negative case {i}:")
    print(negative_cases[i])
    print(gogame.str(negative_cases[i], nn_format=True))
    print()

In [10]:
# Probing
BINARY = True
epochs = 50

In [8]:
# First test if the concept can be regressed form the inputs
name = "input"
all_cases = np.concatenate([positive_cases, negative_cases])
all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
all_labels = np.array(all_labels)
shuffled_indices = np.arange(all_labels.shape[0])

np.random.shuffle(shuffled_indices)

all_cases = all_cases[shuffled_indices]
all_labels = all_labels[shuffled_indices]

points = all_cases.reshape(all_cases.shape[0], -1)

POSITIONS_TO_CONSIDER = points.shape[0]

print("Positions to consider: ", POSITIONS_TO_CONSIDER)

# Use the regression
score = linear_regression.perform_regression(
    points=points[:POSITIONS_TO_CONSIDER], 
    targets=all_labels[:POSITIONS_TO_CONSIDER], 
    validation_points=points[POSITIONS_TO_CONSIDER:], 
    validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
    is_binary=BINARY,
    epochs=epochs,
    dynamic=True
)

score = 0 if score < 0 else score

print("Regression score: ", score)

concept_folder_setup_and_score('dynamic', model_type, board_name, session_name, concept_name, name, score)

Positions to consider:  38


2024-02-20 19:38:37.848252: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


2/2 [==============================] - 0s 9ms/step
-0.052631578947368474
Regression score:  0


In [11]:
epochs_to_look_at = agents_to_sample

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    if resnet:
        model = ResNet(board_size, path)
    else:
        model = ConvNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_activation_values(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = linear_regression.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=BINARY,
            epochs=epochs,
            dynamic=True
        )
        score = 0 if score < 0 else score
        concept_presence_per_layer.append(score)

        print(f"The presence of {concept_name} in resblock {i} is {score}")
    
    concept_presences[concept_name] = concept_presence_per_layer

    concept_folder_setup_and_score('dynamic', model_type, board_name, session_name, concept_name, epoch, concept_presence_per_layer)

Getting activation outputs: 100%|██████████| 2/2 [00:00<00:00, 31.77it/s]


Performing regression for layer 0


2024-02-20 19:41:33.505657: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


2/2 [==============================] - 0s 11ms/step
0.42105263157894735
The presence of ladder in resblock 0 is 0.42105263157894735
Performing regression for layer 1


2024-02-20 19:41:36.767594: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


...

2/2 [==============================] - 0s 15ms/step
0.5789473684210527
The presence of ladder in resblock 5 is 0.5789473684210527
Performing regression for layer 6


2024-02-20 19:44:10.892334: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


2/2 [==============================] - 0s 12ms/step
0.5263157894736843
The presence of ladder in resblock 6 is 0.5263157894736843
Performing regression for layer 7


2024-02-20 19:44:14.182955: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


2/2 [==============================] - 0s 13ms/step
-0.052631578947368474
The presence of ladder in resblock 7 is 0
